##This notebook combines the two datasets of locusts created in other notebooks and then trains two models on it - RandomForest and SVC classifier

In [3]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
import time
from sklearn import metrics
import pickle
import numpy as np
import requests
import os
import pandas as pd
import csv

from io import BytesIO
from matplotlib.pyplot import imshow
from PIL import Image

import rasterio
import rasterio.features
import rasterio.warp
from rasterio import MemoryFile
from PIL import ImageColor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [2]:
!pip install rasterio


     |████████████████████████████████| 18.1MB 1.3MB/s 


In [5]:
df1 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Train_Data.csv")
df2 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Train_Data _Random.csv")

In [6]:
df2.head()

,Unnamed: 0,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Elevation,Soil_Temperature,Vegetation_Index
0,0,47.668856,-117.863491,22258,6/30/2020,0,#128800,#bfdf0c,#f1fc05,#338301
1,1,25.359223,-70.869351,22259,6/30/2020,0,#182033,#182033,#182033,#182033
2,2,34.606546,-76.413211,22260,6/30/2020,0,#223543,#223543,#223543,#223543
3,3,39.889036,-101.434191,22261,6/30/2020,0,#fffa00,#c5d40b,#ff9800,#87b30a
4,4,44.563030,-94.805502,22262,6/30/2020,0,#00e9d3,#15b446,#ffa300,#96ba20


In [7]:
df1.head()

,Unnamed: 0,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Elevation,Soil_Temperature,Vegetation_Index
0,0,44.073889,9.503333,34132,2020-06-30,1,#fffa00,#ccc80a,#ff4d00,#b09279
1,1,45.466944,9.577222,34133,2020-06-30,1,#fff900,#c6d20b,#ff5100,#b0cf69
2,2,45.466389,9.576111,34134,2020-06-30,1,#fff900,#c6d20b,#ff5100,#9a6e5a
3,3,37.425833,3.141667,34149,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab
4,4,37.425833,3.141667,34150,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab


In [8]:
df1

,Unnamed: 0,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Elevation,Soil_Temperature,Vegetation_Index
0,0,44.073889,9.503333,34132,2020-06-30,1,#fffa00,#ccc80a,#ff4d00,#b09279
1,1,45.466944,9.577222,34133,2020-06-30,1,#fff900,#c6d20b,#ff5100,#b0cf69
2,2,45.466389,9.576111,34134,2020-06-30,1,#fff900,#c6d20b,#ff5100,#9a6e5a
3,3,37.425833,3.141667,34149,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab
4,4,37.425833,3.141667,34150,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab
...,...,...,...,...,...,...,...,...,...,...
345,345,35.037500,2.606667,36638,2020-06-30,1,#fffb00,#c1db0b,#ff3f00,#bfde78
346,346,35.037778,2.606944,36639,2020-06-30,1,#fffb00,#c1db0b,#ff3f00,#bfde77
347,347,37.416667,3.121389,36640,2020-06-30,1,#ffd700,#28b93f,#d50000,#b9ab9c
348,348,36.721667,2.751667,36651,2020-06-30,1,#ffde00,#2bba3e,#c80000,#ddc9bd


In [10]:
df = df1.append(df2)
df

,Unnamed: 0,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Elevation,Soil_Temperature,Vegetation_Index,Latitude,Longitude
0,0,44.073889,9.503333,34132,2020-06-30,1,#fffa00,#ccc80a,#ff4d00,#b09279,NaN,NaN
1,1,45.466944,9.577222,34133,2020-06-30,1,#fff900,#c6d20b,#ff5100,#b0cf69,NaN,NaN
2,2,45.466389,9.576111,34134,2020-06-30,1,#fff900,#c6d20b,#ff5100,#9a6e5a,NaN,NaN
3,3,37.425833,3.141667,34149,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab,NaN,NaN
4,4,37.425833,3.141667,34150,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
345,345,NaN,NaN,22552,6/30/2020,0,#00feff,#00ac4c,#ffaa00,#a9896f,39.697178,-85.876508
346,346,NaN,NaN,22553,6/30/2020,0,#b2d800,#c0dd0b,#ff8c00,#965a46,37.814041,-100.164967
347,347,NaN,NaN,22554,6/30/2020,0,#359a00,#bfde0b,#ffc000,#529800,47.051322,-103.500604
348,348,NaN,NaN,22555,6/30/2020,0,#d5d3d6,#d5d3d6,#d5d3d6,#d5d3d6,30.759271,-70.600521


In [11]:
df.shape

(700, 12)

In [15]:
dataX = df[df.columns[[6,7,8,9]]]
dataY = df[df.columns[[5]]]

In [16]:
dataX.head()

,Soil_Moisture,Elevation,Soil_Temperature,Vegetation_Index
0,#fffa00,#ccc80a,#ff4d00,#b09279
1,#fff900,#c6d20b,#ff5100,#b0cf69
2,#fff900,#c6d20b,#ff5100,#9a6e5a
3,#ffd700,#2bba3e,#d50000,#d1c0ab
4,#ffd700,#2bba3e,#d50000,#d1c0ab


Converting the hex code to rgb components and storing them

In [21]:
count = 0
r,g,b,r1, g1, b1,r2, g2, b2,r3, g3, b3 = [],[],[],[],[],[],[],[],[],[],[],[]
for index, row in dataX.iterrows():
  print(count)
  hex = dataX.iloc[count][0]
  rgb =  ImageColor.getcolor(str(hex), "RGB")
  r.append(rgb[0])
  g.append(rgb[1])
  b.append(rgb[2])
  hex = dataX.iloc[count][1]
  rgb =  ImageColor.getcolor(str(hex), "RGB")
  r1.append(rgb[0])
  g1.append(rgb[1])
  b1.append(rgb[2])
  hex = dataX.iloc[count][2]
  rgb =  ImageColor.getcolor(str(hex), "RGB")
  r2.append(rgb[0])
  g2.append(rgb[1])
  b2.append(rgb[2])
  hex = dataX.iloc[count][3]
  rgb =  ImageColor.getcolor(str(hex), "RGB")
  r3.append(rgb[0])
  g3.append(rgb[1])
  b3.append(rgb[2])
  count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
len(r1)

700

In [23]:
new_columns = ["R_soil", "G_soil", "B_soil", "R_ele", "G_ele", "B_ele","R_temp", "G_temp", "B_temp", "R_veg", "G_veg", "B_veg",]

In [24]:
df[new_columns[0]] = r
df[new_columns[1]] = g
df[new_columns[2]] = b
df[new_columns[3]] = r1
df[new_columns[4]] = g1
df[new_columns[5]] = b1
df[new_columns[6]] = r2
df[new_columns[7]] = g2
df[new_columns[8]] = b2
df[new_columns[9]] = r3
df[new_columns[10]] = g3
df[new_columns[11]] = b3

In [25]:
df.head()

,Unnamed: 0,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Elevation,Soil_Temperature,Vegetation_Index,Latitude,Longitude,R_soil,G_soil,B_soil,R_ele,G_ele,B_ele,R_temp,G_temp,B_temp,R_veg,G_veg,B_veg
0,0,44.073889,9.503333,34132,2020-06-30,1,#fffa00,#ccc80a,#ff4d00,#b09279,NaN,NaN,255,250,0,204,200,10,255,77,0,176,146,121
1,1,45.466944,9.577222,34133,2020-06-30,1,#fff900,#c6d20b,#ff5100,#b0cf69,NaN,NaN,255,249,0,198,210,11,255,81,0,176,207,105
2,2,45.466389,9.576111,34134,2020-06-30,1,#fff900,#c6d20b,#ff5100,#9a6e5a,NaN,NaN,255,249,0,198,210,11,255,81,0,154,110,90
3,3,37.425833,3.141667,34149,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab,NaN,NaN,255,215,0,43,186,62,213,0,0,209,192,171
4,4,37.425833,3.141667,34150,2020-06-30,1,#ffd700,#2bba3e,#d50000,#d1c0ab,NaN,NaN,255,215,0,43,186,62,213,0,0,209,192,171


In [27]:
X = df[df.columns[[12,13,14,15,16,17,18,19,20,21,22,23]]]
X

,R_soil,G_soil,B_soil,R_ele,G_ele,B_ele,R_temp,G_temp,B_temp,R_veg,G_veg,B_veg
0,255,250,0,204,200,10,255,77,0,176,146,121
1,255,249,0,198,210,11,255,81,0,176,207,105
2,255,249,0,198,210,11,255,81,0,154,110,90
3,255,215,0,43,186,62,213,0,0,209,192,171
4,255,215,0,43,186,62,213,0,0,209,192,171
...,...,...,...,...,...,...,...,...,...,...,...,...
345,0,254,255,0,172,76,255,170,0,169,137,111
346,178,216,0,192,221,11,255,140,0,150,90,70
347,53,154,0,191,222,11,255,192,0,82,152,0
348,213,211,214,213,211,214,213,211,214,213,211,214


In [28]:
dataY.astype(int)

,Locust
0,1
1,1
2,1
3,1
4,1
...,...
345,0
346,0
347,0
348,0


Train-Test Split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, dataY, test_size=0.3)

Random Forest Classifier

In [37]:
clf=RandomForestClassifier(n_estimators=100)

In [38]:
X_train

,R_soil,G_soil,B_soil,R_ele,G_ele,B_ele,R_temp,G_temp,B_temp,R_veg,G_veg,B_veg
321,255,249,0,198,210,11,255,81,0,176,207,105
210,178,216,0,65,191,54,255,159,0,83,151,0
283,86,100,113,86,100,113,86,100,113,86,100,113
287,255,219,0,78,195,50,204,0,0,200,181,165
69,54,61,77,54,61,77,54,61,77,54,61,77
...,...,...,...,...,...,...,...,...,...,...,...,...
51,17,23,39,17,23,39,17,23,39,17,23,39
265,255,251,0,193,219,11,255,63,0,191,222,120
68,255,205,0,0,162,69,255,92,0,209,192,172
74,255,251,0,192,220,11,255,63,0,157,124,95


In [39]:
y_train = list(y_train[y_train.columns[0]])
y_test = list(y_test[y_test.columns[0]])

In [40]:
clf = clf.fit(X_train,y_train)

In [41]:
y_pred = clf.predict(X_test)

In [42]:
print("Training Accuracy:",metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred))
base_accuracy = metrics.accuracy_score(y_test, y_pred)

Training Accuracy: 1.0
Test Accuracy: 1.0


In [43]:
cm = metrics.confusion_matrix(y_test,y_pred)

In [44]:
cm

array([[106,   0],
       [  0, 104]])

Hyperparameter Tuning

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 110, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4, 10]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 16, 28, 40, 51, 63, 75, 86, 98, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 10], 'bootstrap': [True, False]}


In [ ]:
start_time = time.time()
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
print("Total time taken--- %s seconds---" % (time.time() - start_time))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   49.5s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.7min finished


Total time taken--- 345.847177028656 seconds---


In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy = metrics.accuracy_score(test_labels, predictions)
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy, predictions

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy, preds = evaluate(best_random, X_test, y_test)
random_accuracy

Model Performance
Accuracy = 0.69%.


0.6865671641791045

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of 2.22%.


In [ ]:
cm = metrics.confusion_matrix(y_test,preds)
cm

array([[24, 30],
       [12, 68]])

In [45]:
pickle.dump(clf, open("best_rf_model.sav", 'wb'))

**Support** **Vector** **Machines**

In [46]:
svc = svm.SVC(kernel='rbf') 

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print(accuracy)

1.0


In [47]:
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 1.0
Recall: 1.0


In [48]:
print("Train accuracy", metrics.accuracy_score(y_train, svc.predict(X_train)))
print("Test accuracy", accuracy)
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))

Train accuracy 0.9938775510204082
Test accuracy 1.0
[[106   0]
 [  0 104]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       106
           1       1.00      1.00      1.00       104

    accuracy                           1.00       210
   macro avg       1.00      1.00      1.00       210
weighted avg       1.00      1.00      1.00       210



Hyperparameter tuning

In [ ]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

In [ ]:
start_time = time.time()
svm = svm.SVC()
grid = GridSearchCV(svm,param_grid,verbose=2, cv = 3)
grid.fit(X_train,y_train)
print("Total time taken--- %s seconds---" % (time.time() - start_time))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


In [ ]:
print(grid.best_estimator_)
best_svm = grid.best_estimator_

In [ ]:
#pickle.dump(clf, open("rf_model.sav", 'wb'))

In [49]:
pickle.dump(svc, open("BEST_SVM_model.sav", 'wb'))

**Python API function to use this model**

In [50]:
feature_list = ['Soil_Moisture', 'Elevation', 'Soil_Temperature', 'Vegetation_Index']

In [51]:
List_URL = [
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Analyzed_Surface_Soil_Moisture&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,ASTER_GDEM_Color_Index&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Soil_Temperature_Layer_1&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,MODIS_Terra_NDVI_8Day&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
]

#1:SMAP_L4_Analyzed_Surface_Soil_Moisture
#2:ASTER_GDEM_Color_Index (Elevation)
#3:SMAP_L4_Soil_Temperature_Layer_1 (Soil Temp)
#4 MODIS_Terra_NDVI_8Day

In [103]:
def get_val(lat, longo, time):
  unit_length = 0.06
  lower_lat = lat - unit_length
  left_long = longo - unit_length

  upper_lat = lat + unit_length
  right_long = longo + unit_length 

  Bbox = (lower_lat,left_long,upper_lat,right_long)

  Bbox = str(Bbox)[1:]

  Bbox = Bbox[:-1]

  Bbox = Bbox.replace(" ", "")
  df = pd.DataFrame(columns = feature_list)
  for i in range(len(List_URL)):
      
      link = List_URL[i]
      url = link.format(time, Bbox)
      response = requests.get(url)
      # print(url)
      img = BytesIO(response.content)

      with open("example2.tif", "wb") as  f:
        f.write(img.getvalue())

      with rasterio.open('example2.tif') as dataset:
        mask = dataset.dataset_mask()
        
        for geom, val in rasterio.features.shapes(
                mask, transform=dataset.transform):
  
            geom = rasterio.warp.transform_geom(
              dataset.crs, 'EPSG:4326', geom, precision=6)
     
            b = dataset.index(longo, lat)
            a = dataset.read()
            index = a[:,b[0]-1,b[1]]
            index = '#%02x%02x%02x' % (index[0], index[1],index[2])
            
            df.loc[0, feature_list[i]] = index
  return df

In [53]:
new_columns = ["R_soil", "G_soil", "B_soil", "R_ele", "G_ele", "B_ele","R_temp", "G_temp", "B_temp", "R_veg", "G_veg", "B_veg",]

In [112]:
def getRBGValues(df):
  count = 0
  r,g,b,r1, g1, b1,r2, g2, b2,r3, g3, b3 = [],[],[],[],[],[],[],[],[],[],[],[]
  for index, row in df.iterrows():
    # print(count)
    hex = df.iloc[count][0]
    # print("1",df)
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r.append(rgb[0])
    g.append(rgb[1])
    b.append(rgb[2])
    # print("2",hex)
    hex = df.iloc[count][1]
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r1.append(rgb[0])
    g1.append(rgb[1])
    b1.append(rgb[2])
    hex = df.iloc[count][2]
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r2.append(rgb[0])
    g2.append(rgb[1])
    b2.append(rgb[2])
    hex = df.iloc[count][3]
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r3.append(rgb[0])
    g3.append(rgb[1])
    b3.append(rgb[2])
    count += 1
    df[new_columns[0]] = r
  df[new_columns[1]] = g
  df[new_columns[2]] = b
  df[new_columns[3]] = r1
  df[new_columns[4]] = g1
  df[new_columns[5]] = b1
  df[new_columns[6]] = r2
  df[new_columns[7]] = g2
  df[new_columns[8]] = b2
  df[new_columns[9]] = r3
  df[new_columns[10]] = g3
  df[new_columns[11]] = b3
  return df

In [59]:
def load_model():
  model = pickle.load(open("best_rf_model.sav", 'rb'))
  return model

Just call this function with latitude, longitude and date to check for locust invasion

In [107]:
def get_locust_prob(lat, longo, date):
  df = get_val(lat, longo, date)
  df = getRBGValues(df)
  model = load_model()  
  prediction = model.predict(df[df.columns[4:]])
  print("predcition is",prediction)
  return prediction

API test

In [99]:
lat = 9.5033
longo = 44.073
date = "2020-06-30"
get_locust_prob(lat, longo, date)

https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=9.443299999999999,44.013,9.5633,44.133&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Analyzed_Surface_Soil_Moisture&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=9.443299999999999,44.013,9.5633,44.133&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,ASTER_GDEM_Color_Index&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=9.443299999999999,44.013,9.5633,44.133&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Soil_Temperature_Layer_1&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=9.443299999999999,44.013,9.5633,44.133&C

array([1])

In [100]:
lat = -80.920
longo = 49.2727
date = "2020-06-30"
get_locust_prob(lat, longo, date)

https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=-80.98,49.2127,-80.86,49.3327&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Analyzed_Surface_Soil_Moisture&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=-80.98,49.2127,-80.86,49.3327&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,ASTER_GDEM_Color_Index&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=-80.98,49.2127,-80.86,49.3327&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Soil_Temperature_Layer_1&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2020-06-30&BBOX=-80.98,49.2127,-80.86,49.3327&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_Corre

array([0])

In [77]:
kenya = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Kenya_Coordinates.csv")

In [78]:
kenya.head()

,Row,Latitude,Longitude,Value
0,0,2.890300,37.314312,-1
1,1,2.439163,38.226922,-1
2,2,-3.230316,35.183446,-1
3,3,3.248599,39.018753,-1
4,4,0.153488,40.575585,-1


In [80]:
infestion_date = "2020-06-25"
post_date = "2020-09-16"

In [114]:
def mulitple_locust_prob(df, date):
  values = []
  count = 0
  for row in df.iterrows():
    if(count > 150): break
    lat = df.iloc[count][1]
    longo = df.iloc[count][2]
    pred = get_locust_prob(lat, longo, date)
    print(count)
    values.append(pred[0])
    count +=1
  return values

In [115]:
val = mulitple_locust_prob(kenya, infestion_date)

predcition is [1]
0
predcition is [1]
1
predcition is [0]
2
predcition is [0]
3
predcition is [1]
4
predcition is [1]
5
predcition is [1]
6
predcition is [1]
7
predcition is [0]
8
predcition is [1]
9
predcition is [0]
10
predcition is [0]
11
predcition is [0]
12
predcition is [0]
13
predcition is [1]
14
predcition is [0]
15
predcition is [1]
16
predcition is [0]
17
predcition is [0]
18
predcition is [0]
19
predcition is [1]
20
predcition is [0]
21
predcition is [0]
22
predcition is [0]
23
predcition is [0]
24
predcition is [0]
25
predcition is [1]
26
predcition is [0]
27
predcition is [0]
28
predcition is [0]
29
predcition is [0]
30
predcition is [0]
31
predcition is [0]
32
predcition is [0]
33
predcition is [0]
34
predcition is [0]
35
predcition is [0]
36
predcition is [0]
37
predcition is [0]
38
predcition is [0]
39
predcition is [1]
40
predcition is [0]
41
predcition is [0]
42
predcition is [0]
43
predcition is [0]
44
predcition is [0]
45
predcition is [0]
46
predcition is [1]
47
pr

In [118]:
new_kenya = kenya.head(151)

In [120]:
new_kenya["InfestationDatePrediction"] = val
new_kenya.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Row,Latitude,Longitude,Value,InfestationDatePrediction
0,0,2.890300,37.314312,-1,1
1,1,2.439163,38.226922,-1,1
2,2,-3.230316,35.183446,-1,0
3,3,3.248599,39.018753,-1,0
4,4,0.153488,40.575585,-1,1


In [121]:
val2 = mulitple_locust_prob(kenya, post_date)

predcition is [0]
0
predcition is [0]
1
predcition is [0]
2
predcition is [0]
3
predcition is [0]
4
predcition is [1]
5
predcition is [1]
6
predcition is [0]
7
predcition is [0]
8
predcition is [0]
9
predcition is [0]
10
predcition is [1]
11
predcition is [0]
12
predcition is [0]
13
predcition is [0]
14
predcition is [0]
15
predcition is [0]
16
predcition is [0]
17
predcition is [0]
18
predcition is [0]
19
predcition is [1]
20
predcition is [0]
21
predcition is [0]
22
predcition is [0]
23
predcition is [0]
24
predcition is [0]
25
predcition is [1]
26
predcition is [0]
27
predcition is [1]
28
predcition is [1]
29
predcition is [0]
30
predcition is [0]
31
predcition is [1]
32
predcition is [0]
33
predcition is [0]
34
predcition is [1]
35
predcition is [0]
36
predcition is [1]
37
predcition is [0]
38
predcition is [1]
39
predcition is [0]
40
predcition is [1]
41
predcition is [0]
42
predcition is [0]
43
predcition is [0]
44
predcition is [1]
45
predcition is [1]
46
predcition is [1]
47
pr

In [122]:
new_kenya["NonInfestationDatePrediction"] = val2
new_kenya.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Row,Latitude,Longitude,Value,InfestationDatePrediction,NonInfestationDatePrediction
0,0,2.890300,37.314312,-1,1,0
1,1,2.439163,38.226922,-1,1,0
2,2,-3.230316,35.183446,-1,0,0
3,3,3.248599,39.018753,-1,0,0
4,4,0.153488,40.575585,-1,1,0


In [123]:
new_kenya.to_csv("new_kenya.csv")